In [1]:
import tensorflow as tf
from utils import cifar10_load
from models import build_ResNet
from tensorflow.keras.models import Model


(x_train, y_train), (x_dev, y_dev), (x_test, y_test) = cifar10_load()

# 这里用name_scope是无效的
model = tf.keras.models.load_model('ResNet32_acc893.h5')
base_varlist = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [2]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

lr_init 0.1, 40epochs后调为0.01, 60epochs后调为0.001, 共100epochs

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Sequential

# 这里name_scope是有效的
detector = Sequential([
    Conv2D(96, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal', input_shape=(32,32,16)),
    MaxPooling2D(),
    Conv2D(192, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D(),
    Conv2D(192, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal'),
    Conv2D(2, (1,1), strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal'),
    GlobalAveragePooling2D()])
detect_pipeline = Model(inputs=model.input, outputs=Activation('softmax', name="detect_softmax")(detector(model.get_layer('activation_10').output)))

In [4]:
from cleverhans.utils_keras import KerasModelWrapper
from modified_PGD import ProjectedGradientDescent
import numpy as np

x = tf.placeholder(tf.float32, shape=(None,32,32,3))
x_label = tf.placeholder(tf.float32, shape=(None,2))
adv_label = tf.placeholder(tf.float32, shape=(None,2))

model_wrap = KerasModelWrapper(model)
detect_wrap = KerasModelWrapper(detect_pipeline)
pgd_wrap = ProjectedGradientDescent(model_wrap, detect_wrap)

pgd_params = {'eps':0.3, 'eps_iter':0.25, 'nb_iter':10, 'y':adv_label, 'ord':np.inf,
              'clip_min':0, 'clip_max':1, 'y_target':None, 'rand_init':None, 'rand_minmax':0.3, 'sanity_checks':True}

adv_x = pgd_wrap.generate(x, **pgd_params)

/tf/robustnn_workspace/cleverhans-master/cleverhans/attacks_tf.py:27: UserWarning: attacks_tf is deprecated and will be removed on 2019-07-18 or after. Code should import functions from their new locations directly.
  warnings.warn("attacks_tf is deprecated and will be removed on 2019-07-18"


Instructions for updating:
Use tf.cast instead.


/tf/robustnn_workspace/cleverhans-master/cleverhans/compat.py:23: UserWarning: <function reduce_max_v1 at 0x7f722d1bbea0> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "


Instructions for updating:
dim is deprecated, use axis instead


/tf/robustnn_workspace/cleverhans-master/cleverhans/compat.py:23: UserWarning: <function reduce_sum_v1 at 0x7f722d1bb1e0> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "


In [5]:
def generate_labels(length):
    adv_labels = np.zeros([length, 2])
    normal_labels = np.copy(adv_labels)
    adv_labels[:, 1] = 1 
    normal_labels[:, 0] = 1
    
    return adv_labels, normal_labels

In [6]:
def initialize_uninitialized(sess):
       global_vars = tf.global_variables()
       is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
       not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]
       #print([str(i.name) for i in not_initialized_vars]) # only for testing
       if len(not_initialized_vars):
            sess.run(tf.variables_initializer(not_initialized_vars))

In [7]:
total_varlist = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
new_varlist = list(set(total_varlist) - set(base_varlist))

In [8]:
batch_size = 32
training_epochs = 100

detect_x = detect_pipeline(x)
detect_adv = detect_pipeline(adv_x)

sess = tf.keras.backend.get_session()

loss1 = tf.keras.backend.categorical_crossentropy(x_label, detect_x)
loss2 = tf.keras.backend.categorical_crossentropy(adv_label, detect_adv)
total_loss = loss1 + loss2
train_step = tf.train.AdamOptimizer(1e-4).minimize(total_loss, var_list=new_varlist)

correct_prediction1 = tf.equal(tf.argmax(detect_x, 1), tf.argmax(x_label, 1))
correct_prediction2 = tf.equal(tf.argmax(detect_adv, 1), tf.argmax(adv_label, 1))
accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, "float"), name='accuracy')
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, "float"), name='accuracy')
accuracy = 0.5*accuracy1 + 0.5*accuracy2

initialize_uninitialized(sess)

['beta1_power:0', 'beta2_power:0', 'conv2d_2_1/bias/Adam:0', 'conv2d_2_1/bias/Adam_1:0', 'conv2d_3_1/bias/Adam:0', 'conv2d_3_1/bias/Adam_1:0', 'conv2d_1_1/bias/Adam:0', 'conv2d_1_1/bias/Adam_1:0', 'conv2d_2_1/kernel/Adam:0', 'conv2d_2_1/kernel/Adam_1:0', 'conv2d_33/kernel/Adam:0', 'conv2d_33/kernel/Adam_1:0', 'conv2d_1_1/kernel/Adam:0', 'conv2d_1_1/kernel/Adam_1:0', 'conv2d_33/bias/Adam:0', 'conv2d_33/bias/Adam_1:0', 'conv2d_3_1/kernel/Adam:0', 'conv2d_3_1/kernel/Adam_1:0']


In [9]:
for i in range(training_epochs):
    adv_labels, normal_labels = generate_labels(batch_size)
    for j in range(len(x_train)//batch_size-1):
        train_step.run(session=sess, feed_dict={x: x_train[j*batch_size:(j+1)*batch_size],
                                               x_label: normal_labels, adv_label: adv_labels})
    adv_labels, normal_labels = generate_labels(len(x_dev[:500]))
    print("iteration: %d, test accuracy: %g" % (i, accuracy.eval(session=sess, feed_dict={x: x_dev[:500], 
                                                                                          x_label: normal_labels, 
                                                                                          adv_label: adv_labels})))

iteration: 0, test accuracy: 0.993
iteration: 1, test accuracy: 1
iteration: 2, test accuracy: 1
iteration: 3, test accuracy: 1
iteration: 4, test accuracy: 1
iteration: 5, test accuracy: 1
iteration: 6, test accuracy: 1
iteration: 7, test accuracy: 1
iteration: 8, test accuracy: 1
iteration: 9, test accuracy: 1
iteration: 10, test accuracy: 1
iteration: 11, test accuracy: 1
iteration: 12, test accuracy: 1
iteration: 13, test accuracy: 1
iteration: 14, test accuracy: 1
iteration: 15, test accuracy: 1
iteration: 16, test accuracy: 1
iteration: 17, test accuracy: 1
iteration: 18, test accuracy: 1
iteration: 19, test accuracy: 1
iteration: 20, test accuracy: 1
iteration: 21, test accuracy: 1
iteration: 22, test accuracy: 1
iteration: 23, test accuracy: 1
iteration: 24, test accuracy: 1
iteration: 25, test accuracy: 1


KeyboardInterrupt: 

In [10]:
detect_pipeline.save('detector.h5')

In [3]:
outlayers = [layer.get_output_at(-1) for layer in detect_pipeline.layers]
test = Model(inputs=detect_pipeline.input, outputs=outlayers)

将Dectector和原来model的隐藏层连起来

In [3]:
detect_pipeline.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization_v1[0][0]     
__________________________________________________________________________________________________
conv2d_1 (

In [3]:
from cleverhans.utils_keras import KerasModelWrapper

x = tf.placeholder(tf.float32, shape=(None,32,32,3))
y = tf.placeholder(tf.float32, shape=(None,10))
model_wrap = KerasModelWrapper(model)

In [2]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Sequential

hidden = model.get_layer('activation_10').output

conv1 = Conv2D(96, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='uniform',
              name='detector_conv1')(hidden)
mp1 = MaxPooling2D(name='detector_mp1')(conv1)
conv2 = Conv2D(192, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='uniform',
              name='detector_conv2')(mp1)
mp2 = MaxPooling2D(name='detector_mp2')(conv2)
conv3 = Conv2D(192, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='uniform',
              name='detector_conv3')(mp2)
conv4 = Conv2D(2, (1,1), strides=(1,1), padding='same', activation='relu', kernel_initializer='uniform',
              name='detector_conv4')(conv3)
GAP = GlobalAveragePooling2D(name='detector_GAP')(conv4)

detector_out = Activation('softmax', name='detector_softmax')(GAP)
detect_pipeline = Model(inputs=model.input, outputs=detector_out)

In [4]:
for layer in model.layers:
    layer.trainable = False

detect_pipeline = Model(inputs=model.input, outputs=Activation('softmax', name="detect_softmax")(detector(model.get_layer('add_3').output)))
detect_wrap = KerasModelWrapper(detect_pipeline)

In [16]:
detect_pipeline.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization_v1[0][0]     
__________________________________________________________________________________________________
conv2d_1 (

In [3]:
from utils import cifar10_load
(x_train, y_train), (x_dev, y_dev), (x_test, y_test) = cifar10_load()
import numpy as np
detect_pipeline.predict(x_dev[0].reshape(-1,32,32,3))

Using TensorFlow backend.


array([[0.49706942, 0.5029305 ]], dtype=float32)

In [15]:
detect_pipeline.save('pipeline_test.h5')

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
detect_pipeline = tf.keras.models.load_model('pipeline_test.h5')
outlayers = [layer.output for layer in detect_pipeline.layers]

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
outlayers[31]

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(?, 2) dtype=float32>

In [15]:
test = Model(inputs=detect_pipeline.input, outputs=outlayers[32])

检测器对对抗样本输出为1,正常样本输出为0

In [14]:
from modified_PGD import ProjectedGradientDescent
import numpy as np

pgd_wrap = ProjectedGradientDescent(model_wrap, detect_wrap)
pgd_params = {'eps':0.3, 'eps_iter':0.25, 'nb_iter':10, 'y':np.array([[0, 1]]), 'ord':np.inf,
              'clip_min':0, 'clip_max':1, 'y_target':None, 'rand_init':None, 'rand_minmax':0.3, 'sanity_checks':True}
adv_x = pgd_wrap.generate(x, **pgd_params)

#train_pipeline = Model(inputs=x, outputs=[model(x), model(adv_x), detect_pipeline(x), detect_pipline(adv_x)])

/tf/robustnn_workspace/cleverhans-master/cleverhans/attacks_tf.py:27: UserWarning: attacks_tf is deprecated and will be removed on 2019-07-18 or after. Code should import functions from their new locations directly.
  warnings.warn("attacks_tf is deprecated and will be removed on 2019-07-18"


Instructions for updating:
Use tf.cast instead.


/tf/robustnn_workspace/cleverhans-master/cleverhans/compat.py:23: UserWarning: <function reduce_max_v1 at 0x7f87a1f6bea0> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "


Instructions for updating:
dim is deprecated, use axis instead


/tf/robustnn_workspace/cleverhans-master/cleverhans/compat.py:23: UserWarning: <function reduce_sum_v1 at 0x7f87a1f6b1e0> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "


ValueError: Graph disconnected: cannot obtain value for tensor Tensor("conv2d_input:0", shape=(?, 32, 32, 16), dtype=float32) at layer "conv2d_input". The following previous layers were accessed without issue: ['input_1', 'conv2d', 'batch_normalization_v1', 'activation', 'conv2d_1', 'batch_normalization_v1_1', 'activation_1', 'conv2d_2', 'batch_normalization_v1_2', 'add', 'activation_2', 'conv2d_3', 'batch_normalization_v1_3', 'activation_3', 'conv2d_4', 'batch_normalization_v1_4', 'add_1', 'activation_4', 'conv2d_5', 'batch_normalization_v1_5', 'activation_5', 'conv2d_6', 'batch_normalization_v1_6', 'add_2', 'activation_6']

train pipeline没有办法抽象成Model储存，只能手写训练流程

In [7]:
model.evaluate(x_dev, y_dev)

5000/5000 [==============================] - 2s 483us/sample - loss: 0.4888 - acc: 0.8930


[0.4887514706611633, 0.893]

In [12]:
hidden = model_wrap.get_layer(x, 'activation_10')
detector_out = detector(hidden)

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization_v1[0][0]     
__________________________________________________________________________________________________
conv2d_1 (

In [6]:
detector.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 96)        13920     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 192)       166080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 192)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 192)         331968    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 2)           386       
_________________________________________________________________
global_average_pooling2d (Gl (None, 2)                 0         
Total para

In [34]:
detect_pipeline.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization_v1[0][0]     
__________________________________________________________________________________________________
conv2d_1 (

In [ ]:
with tf.variable_scope('base_model'):
    model = build_ResNet(depth=32)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(x_train)

In [3]:
lr_adjust = [0.1, 0.1, 0.01, 0.001, 0.001]
for lr in lr_adjust:
    model.compile(
        optimizer=tf.keras.optimizers.SGD(lr=lr, momentum=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), steps_per_epoch=len(x_train)//32,
                        validation_data=(x_dev, y_dev), epochs=20)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1407/1407 [==============================] - 42s 30ms/step - loss: 2.4361 - acc: 0.2191 - val_loss: 2.8632 - val_acc: 0.2458
Epoch 2/20
1407/1407 [==============================] - 40s 28ms/step - loss: 1.8852 - acc: 0.3867 - val_loss: 1.8628 - val_acc: 0.4438
Epoch 3/20
1407/1407 [==============================] - 40s 28ms/step - loss: 1.5862 - acc: 0.5012 - val_loss: 1.5584 - val_acc: 0.5220
Epoch 4/20
1407/1407 [==============================] - 40s 28ms/step - loss: 1.4372 - acc: 0.5548 - val_loss: 1.5674 - val_acc: 0.5258
Epoch 5/20
1407/1407 [==============================] - 40s 29ms/step - loss: 1.3488 - acc: 0.5901 - val_loss: 1.6992 - val_acc: 0.5288
Epoch 6/20
1407/1407 [==============================] - 40s 28ms/step - loss: 1.2744 - acc: 0.6233 - val_loss: 1.5964 - val_acc: 0.5820
Epoch 7/20
1407/1407 [==============================] - 40s 28ms/step - loss: 1.2114 - acc: 0.6525 - val_loss: 1.2162 - val_acc: 0.6548


1407/1407 [==============================] - 40s 28ms/step - loss: 0.3642 - acc: 0.9313 - val_loss: 0.5059 - val_acc: 0.8914
Epoch 13/20
1407/1407 [==============================] - 40s 29ms/step - loss: 0.3626 - acc: 0.9308 - val_loss: 0.4978 - val_acc: 0.8938
Epoch 14/20
1407/1407 [==============================] - 40s 28ms/step - loss: 0.3596 - acc: 0.9311 - val_loss: 0.5085 - val_acc: 0.8938
Epoch 15/20
1407/1407 [==============================] - 40s 28ms/step - loss: 0.3572 - acc: 0.9328 - val_loss: 0.5047 - val_acc: 0.8950
Epoch 16/20
1407/1407 [==============================] - 40s 29ms/step - loss: 0.3539 - acc: 0.9330 - val_loss: 0.5080 - val_acc: 0.8928
Epoch 17/20
1407/1407 [==============================] - 41s 29ms/step - loss: 0.3514 - acc: 0.9341 - val_loss: 0.5076 - val_acc: 0.8914
Epoch 18/20
1407/1407 [==============================] - 41s 29ms/step - loss: 0.3467 - acc: 0.9346 - val_loss: 0.5095 - val_acc: 0.8910
Epoch 19/20
1407/1407 [==============================

In [5]:
model.save('ResNet32_acc893.h5')